In [1]:
# Prashant Singh
# 2021CSB1124
import cv2
import dlib
import numpy as np
import math
import imageio.v2 as imageio
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay



In [2]:
def imageMorph(input_pts,output_pts,alpha,input_img,output_img):
    # Final Result of Morphing for given alpha initialised as empty image
    result=np.zeros(input_img.shape,dtype="uint8")
    
    
    for i in range (0,input_pts.shape[0]):
        # Triangle points for input, output and morph triangle
        tri1=input_pts[i]
        tri2=output_pts[i]
        tri_morph=np.zeros((3,2))
        tri_morph=(input_pts[i])*(1-alpha)+(output_pts[i])*alpha
        tri_morph=tri_morph.astype('uint')

        # Bounding triangles into Rectangles to perform Affine Transform
        x,y,w,h=cv2.boundingRect(np.int32(tri1))
        x1,y1,w1,h1=cv2.boundingRect(np.int32(tri2))
        xm,ym,wm,hm=cv2.boundingRect(np.int32(tri_morph))

        # Initialising Input and Output Bounding rectangles
        rect1=input_img[y:y+h,x:x+w]
        rect2=output_img[y1:y1+h1,x1:x1+w1] 
                
        
        # New coordinates of Input,Morph and Output Triangle in Bounding Rectangles for Affine Transform
        tri1_cropped=np.array([[tri1[0][0]-x,tri1[0][1]-y],[tri1[1][0]-x,tri1[1][1]-y],[tri1[2][0]-x,tri1[2][1]-y]])
        tri_cropped_morph=np.array([[tri_morph[0][0]-xm,tri_morph[0][1]-ym],[tri_morph[1][0]-xm,tri_morph[1][1]-ym],[tri_morph[2][0]-xm,tri_morph[2][1]-ym]])
        tri2_cropped=np.array([[tri2[0][0]-x1,tri2[0][1]-y1],[tri2[1][0]-x1,tri2[1][1]-y1],[tri2[2][0]-x1,tri2[2][1]-y1]])
        
        # Affine Transform from Input Rectangle to Morph Rectangle
        M=cv2.getAffineTransform(np.float32(tri1_cropped),np.float32(tri_cropped_morph))
        img1=cv2.warpAffine(rect1,M,(wm,hm),None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101) 
        
        
        # Affine Transform from Output Rectangle to Morph Rectangle
        M1=cv2.getAffineTransform(np.float32(tri2_cropped),np.float32(tri_cropped_morph))
        img2=cv2.warpAffine(rect2,M1,(wm,hm),None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)

        # Creating Mask to Get only triangle out rectangle and making all other values 0 in the rectangle
        mask=np.zeros((hm,wm,3))
        mask=cv2.fillConvexPoly(mask,tri_cropped_morph,color=(1,1,1))
        mask=mask.astype('uint8')
        
        # Alpha Blending of rectangles received by Affine Transforms
        final=img1*(1-alpha)+img2*alpha
        final=final.astype('uint8')

        # Mask is Applied on the Resulting Image by using multiplication with mask and (1-mask)        
        result[ym:ym+hm,xm:xm+wm]=result[ym:ym+hm,xm:xm+wm]*(1-mask)+final*mask
        
    
    
    return result


In [3]:
# Reading Input and Output Images using OpenCV
img = cv2.imread("face1.jpeg")
output_img = cv2.imread("face2.jpeg")

size=(img.shape[1],img.shape[0])

# Resizing the Output Image to the size of Input Image in case both images are not of same size
output_img = cv2.resize(output_img,size,interpolation=cv2.INTER_CUBIC)

# Detector to get list of faces in the image and predictor to get tie points for B part
detector=dlib.get_frontal_face_detector()
predictor=dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
output_gray=cv2.cvtColor(output_img,cv2.COLOR_BGR2GRAY)

# List of Input faces and Output faces is generated
face=detector(gray)
output_face=detector(output_gray)


landmark=predictor(image=gray,box=face[0])
output_landmark=predictor(image=output_gray,box=output_face[0])

# Tie Points for Input and Output Array for Delaunay Triangulation
tie_points=[]
output_tie_points=[]


#Reading Tiepoints from Tiepoints.txt using File Handling 

# f=open('Tiepoints.txt','r')
# t=int(f.readline())

# for i in range (0,t):
#     s=f.readline()
#     lst=s.split()
#     x1=int(lst[0])
#     y1=int(lst[1])
#     x2=int(lst[2])
#     y2=int(lst[3])
#     tie_points.append([x1,y1])
#     output_tie_points.append([x2,y2])
    
# f.close()
       

#       !!!!COMMENT OUT THE FOLLOWING CODE AND COMMENT THE ABOVE CODE OF FILE HANDLING FOR PART B!!!!!

tie_points.clear()
output_tie_points.clear()
for i in range (0,68):
    x=landmark.part(i).x
    y=landmark.part(i).y
    tie_points.append([x,y])
    
    x=output_landmark.part(i).x
    y=output_landmark.part(i).y
    output_tie_points.append([x,y])
    

tie_points.append([0,0])
tie_points.append([0,gray.shape[0]-1])
tie_points.append([gray.shape[1]-1,0])
tie_points.append([img.shape[1]-1,img.shape[0]-1])

output_tie_points.append([0,0])
output_tie_points.append([0,output_gray.shape[0]-1])
output_tie_points.append([output_gray.shape[1]-1,0])
output_tie_points.append([output_img.shape[1]-1,output_img.shape[0]-1])






# Delaunay Triangulation using Inbuilt Function
delaunay=Delaunay(tie_points)
delaunay=delaunay.simplices

delaunay=sorted(delaunay,key=lambda row:(row[0],row[1],row[2]))


n=len(delaunay)
# List containing Tie points corrensponding to each triangle for input and output image
input_pts=np.zeros((n,3,2),dtype="uint")
output_pts=np.zeros((n,3,2),dtype="uint")

for i in range (0,n):
    input_pts[i][0]=tie_points[delaunay[i][0]]
    input_pts[i][1]=tie_points[delaunay[i][1]]
    input_pts[i][2]=tie_points[delaunay[i][2]]     
    
    output_pts[i][0]=output_tie_points[delaunay[i][0]]
    output_pts[i][1]=output_tie_points[delaunay[i][1]]
    output_pts[i][2]=output_tie_points[delaunay[i][2]]
    
    

# Making list of all Intermediate images is stored in num by varying alpha
# Number of Images can be changed by changing num
!mkdir images
images=[]    
    
num=60
alpha=0
increment=1.0/num
for i in range(0,num):
    print(i)
    filename='images/'+str(i)+'.png'
    result=imageMorph(input_pts,output_pts,alpha,img,output_img)
    alpha+=increment
    cv2.imwrite(filename,result)
    images.append(imageio.imread(filename))


# Using Imageio Library to create gif by the name 'result.gif' of images
imageio.mimsave('result.gif',images,duration=0.04)

cv2.destroyAllWindows()

A subdirectory or file images already exists.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
